In [1]:
!pip install pydicom

     |████████████████████████████████| 1.9MB 6.1MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import pandas as pd
import numpy as np
import pydicom
import matplotlib.pyplot as plt
import scipy.misc

In [4]:
cur_path = "/content/drive/MyDrive/Dataset/physionet.org/files/mimic-cxr/2.0.0/"
os.chdir(cur_path)
!pwd

/content/drive/MyDrive/Dataset/physionet.org/files/mimic-cxr/2.0.0


In [5]:
cxr_study_list = pd.read_csv('cxr-study-list.csv')
cxr_record_list = pd.read_csv('cxr-record-list.csv')

In [6]:
cxr_study_list

,subject_id,study_id,path
0,10000032,50414267,files/p10/p10000032/s50414267.txt
1,10000032,53189527,files/p10/p10000032/s53189527.txt
2,10000032,53911762,files/p10/p10000032/s53911762.txt
3,10000032,56699142,files/p10/p10000032/s56699142.txt
4,10000764,57375967,files/p10/p10000764/s57375967.txt
...,...,...,...
227830,19999442,58708861,files/p19/p19999442/s58708861.txt
227831,19999733,57132437,files/p19/p19999733/s57132437.txt
227832,19999987,55368167,files/p19/p19999987/s55368167.txt
227833,19999987,58621812,files/p19/p19999987/s58621812.txt


In [7]:
cxr_record_list

,subject_id,study_id,dicom_id,path
0,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,files/p10/p10000032/s50414267/02aa804e-bde0afd...
1,10000032,50414267,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,files/p10/p10000032/s50414267/174413ec-4ec4c1f...
2,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,files/p10/p10000032/s53189527/2a2277a9-b0ded15...
3,10000032,53189527,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,files/p10/p10000032/s53189527/e084de3b-be89b11...
4,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,files/p10/p10000032/s53911762/68b5c4b1-227d048...
...,...,...,...,...
377105,19999733,57132437,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c,files/p19/p19999733/s57132437/428e2c18-5721d8f...
377106,19999733,57132437,58c403aa-35ff8bd9-73e39f54-8dc9cc5d-e0ec3fa9,files/p19/p19999733/s57132437/58c403aa-35ff8bd...
377107,19999987,55368167,58766883-376a15ce-3b323a28-6af950a0-16b793bd,files/p19/p19999987/s55368167/58766883-376a15c...
377108,19999987,58621812,7ba273af-3d290f8d-e28d0ab4-484b7a86-7fc12b08,files/p19/p19999987/s58621812/7ba273af-3d290f8...


In [8]:
merged = cxr_study_list.merge(cxr_record_list, how='inner', left_on=['subject_id','study_id'], right_on=['subject_id','study_id'])
merged

,subject_id,study_id,path_x,dicom_id,path_y
0,10000032,50414267,files/p10/p10000032/s50414267.txt,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,files/p10/p10000032/s50414267/02aa804e-bde0afd...
1,10000032,50414267,files/p10/p10000032/s50414267.txt,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,files/p10/p10000032/s50414267/174413ec-4ec4c1f...
2,10000032,53189527,files/p10/p10000032/s53189527.txt,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,files/p10/p10000032/s53189527/2a2277a9-b0ded15...
3,10000032,53189527,files/p10/p10000032/s53189527.txt,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,files/p10/p10000032/s53189527/e084de3b-be89b11...
4,10000032,53911762,files/p10/p10000032/s53911762.txt,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,files/p10/p10000032/s53911762/68b5c4b1-227d048...
...,...,...,...,...,...
377105,19999733,57132437,files/p19/p19999733/s57132437.txt,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c,files/p19/p19999733/s57132437/428e2c18-5721d8f...
377106,19999733,57132437,files/p19/p19999733/s57132437.txt,58c403aa-35ff8bd9-73e39f54-8dc9cc5d-e0ec3fa9,files/p19/p19999733/s57132437/58c403aa-35ff8bd...
377107,19999987,55368167,files/p19/p19999987/s55368167.txt,58766883-376a15ce-3b323a28-6af950a0-16b793bd,files/p19/p19999987/s55368167/58766883-376a15c...
377108,19999987,58621812,files/p19/p19999987/s58621812.txt,7ba273af-3d290f8d-e28d0ab4-484b7a86-7fc12b08,files/p19/p19999987/s58621812/7ba273af-3d290f8...


In [9]:
def extract_finding_impression(origin_file):
    fp = open(origin_file)
    lines = fp.readlines() 
    mark = 0
    find_impression = []
    for line in lines:
        # print(line)
        if 'FINDINGS:' in line:
            mark = 1
            continue
        elif 'IMPRESSION:' in line:
            mark = 1
            continue
        elif ('EXAMINATION:' in line) or ('INDICATION:' in line) or ('TECHNIQUE:' in line) or ('COMPARISON:' in line):
            mark = 0
        else:
            if mark == 1:
                find_impression.append(line)

    fp.close()

    find_impression_str = ''
    for i in find_impression:
        i = i.replace('\n', '')
        find_impression_str += i

    # name = origin_file.split('/')[-1]
    # f = open(target_file, 'w+')
    # f.write(find_impression_str)
    # f.close()

    return find_impression_str

In [10]:
pwd

'/content/drive/My Drive/Dataset/physionet.org/files/mimic-cxr/2.0.0'

In [11]:
text_file = 'files/p10/p10000032/s53911762.txt'
content = extract_finding_impression(text_file)
content

'  Single frontal view of the chest provided.  There is no focal consolidation, effusion, or pneumothorax. The cardiomediastinal silhouette is normal.  Again seen are multiple clips projecting over the left breast and remote left-sided rib fractures.  No free air below the right hemidiaphragm is seen.   No acute intrathoracic process.'

In [15]:
merged_head = merged.head(1000)
#merged_head['text_content'] = ''

In [16]:
for index, row in merged_head.iterrows():
    text_file = row['path_x']
    image_raw = row['path_y']
    print(text_file)
    content = extract_finding_impression(text_file)
    print(content)
    #merged_head.loc[index,'text_content'] = content

files/p10/p10000032/s50414267.txt
  There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal.  Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted.   No acute cardiopulmonary process.
files/p10/p10000032/s50414267.txt
  There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal.  Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted.   No acute cardiopulmonary process.
files/p10/p10000032/s53189527.txt
  The cardiac, mediastinal and hilar contours are normal. Pulmonary vasculature is normal.  

KeyboardInterrupt: ignored

In [20]:
paired_data = pd.DataFrame(columns=['image_path', 'text_content'])
sample_count = 0

for index, row in merged_head.iterrows():
    image_raw = row['path_y']
    text_file = row['path_x']

    if (not os.path.exists(image_raw)) or (not os.path.exists(text_file)):
        print('file not exist.')
        continue

    content = extract_finding_impression(text_file)

    # merged.loc[index,'text_content'] = content

    image_out_path = 'image/' + image_raw.split('/')[-1].split('.')[0] + '.jpg'
    ds = pydicom.read_file(image_raw) 
    img = ds.pixel_array 
    # plt.imshow(img)
    # plt.show()
    scipy.misc.imsave(image_out_path,img)

    paired_data.append({'image_path':image_out_path, 'text_content':content},ignore_index=True)

    sample_count += 1
    if sample_count > 10000:
        break
    print(str(sample_count)+' - '+image_out_path)


    

        

file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not exist.
file not